In [58]:
import re
import os
import cv2
import pdb
import glob
import pandas
import argparse
import numpy as np
from tqdm import tqdm
from functools import partial
from multiprocessing import Pool

## Function 

In [137]:
# this function is used to get an path to csv file and then take the data in the csv file put it in a set, the set contains 
# a list elements where each element is a line in csv file

In [138]:
def csv2dict(anno_path, dataset_type):
    inputs_list = pandas.read_csv(anno_path)
    inputs_list = (inputs_list.to_dict()['id|folder|signer|annotation'].values())
    print(type(inputs_list))
    info_dict = dict()
    info_dict['prefix'] = "C:\\Users\LENOVO\Project Final Year\dataset"
    for file_idx, file_info in tqdm(enumerate(inputs_list), total=len(inputs_list)):
        fileid, folder, signer, label = file_info.split("|")
        # print(fileid,'\n', folder,'\n' ,signer,'\n', label)
        num_frames = len(glob.glob(f"{info_dict['prefix']}/{dataset_type}/{folder}"))
        info_dict[file_idx] = {
            'fileid': fileid,
            'folder': f"{dataset_type}/{folder}",
            'signer': signer,
            'label': label,
            'num_frames': num_frames,
            'original_info': file_info,
        }
    return info_dict

## end func

In [139]:
# setting up parser

In [140]:
parser = argparse.ArgumentParser(
    description='Data process for Visual Alignment Constraint for Continuous Sign Language Recognition.')
parser.add_argument('--dataset', type=str, default='phoenix2014',
                    help='save prefix')
parser.add_argument('--dataset-root', type=str, default='../dataset/phoenix2014/phoenix-2014-multisigner',
                    help='path to the dataset')
parser.add_argument('--annotation-prefix', type=str, default='annotations/manual/{}.corpus.csv',
                    help='annotation prefix')
parser.add_argument('--output-res', type=str, default='256x256px',
                    help='resize resolution for image sequence')
parser.add_argument('--process-image', '-p', action='store_true',
                    help='resize image')
parser.add_argument('--multiprocessing', '-m', action='store_true',
                    help='whether adopts multiprocessing to accelate the preprocess')


_StoreTrueAction(option_strings=['--multiprocessing', '-m'], dest='multiprocessing', nargs=0, const=True, default=False, type=None, choices=None, help='whether adopts multiprocessing to accelate the preprocess', metavar=None)

In [141]:
 args = parser.parse_args(args=[])

In [142]:
mode = ["dev", "test", "train"]
sign_dict = dict()
if not os.path.exists(f"./{args.dataset}"):
    os.makedirs(f"./{args.dataset}")
information = csv2dict("C:\\Users\LENOVO\Project Final Year\csv_files\\train_corpus.csv", dataset_type=md)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 668.36it/s]

<class 'dict_values'>


In [136]:
# print(information)

In [128]:
#saving into phoniex file

In [129]:
np.save(f"./{args.dataset}/{md}_info.npy", information)

## Function 

In [143]:
# now we create a dictionary to keep the total no of occourance of gloss labels, we do it by iterating over the dictionary
# and splitting the labels to different words and then appending it over the tot_dict

In [144]:
def sign_dict_update(total_dict, info):
    for k, v in info.items():
        if not isinstance(k, int):
            continue
        split_label = v['label'].split()
        for gloss in split_label:
            if gloss not in total_dict.keys():
                total_dict[gloss] = 1
            else:
                total_dict[gloss] += 1
    return total_dict

## end func

In [145]:
# update the total gloss dict
sign_dict_update(sign_dict, information)
print()

## Function

In [150]:
# this function is used to save the labels, fileid, and signer in a text file

In [151]:
def generate_gt_stm(info, save_path):
    with open(save_path, "w") as f:
        for k, v in info.items():
            if not isinstance(k, int):
                continue
            f.writelines(f"{v['fileid']} 1 {v['signer']} 0.0 1.79769e+308 {v['label']}\n")

## endfunc

In [153]:
# generate groudtruth stm for evaluation
generate_gt_stm(information, f"./{args.dataset}/{args.dataset}-groundtruth-{md}.stm")

In [154]:
video_index = np.arange(len(information) - 1)

## Function

In [172]:
def resize_dataset(video_idx, dsize, info_dict):
    info = info_dict[video_idx]
    img_list = glob.glob(f"{info_dict['prefix']}/{info['folder']}")
    for x in img_list:
        print(x)
    for img_path in img_list:
        img = cv2.imread(img_path)
        rs_img = cv2.resize(img, dsize, interpolation=cv2.INTER_LANCZOS4)
        rs_img_path = img_path.replace("train", "train_new") # here you can make it custom
        rs_img_dir = os.path.dirname(rs_img_path)
        
        if not os.path.exists(rs_img_dir):
            os.makedirs(rs_img_dir)
            cv2.imwrite(rs_img_path, rs_img)
        else:
            cv2.imwrite(rs_img_path, rs_img)

## end func

In [173]:
d_size = (256, 256)
for idx in tqdm(video_index):
    resize_dataset(idx, d_size, information)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000000-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000001-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000002-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000003-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000004-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000005-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-0/1\01April_2010_Thursday_heute.avi_pid0_fn000006-0.png
C:\Users\LENO

 25%|█████████████████████                                                               | 1/4 [00:02<00:08,  2.98s/it]

C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000000-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000001-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000002-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000003-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000004-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000005-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-2/1\01April_2010_Thursday_heute.avi_pid0_fn000006-0.png
C:\Users\LENO

 50%|██████████████████████████████████████████                                          | 2/4 [00:05<00:05,  2.96s/it]

C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000000-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000001-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000002-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000003-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000004-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000005-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-3/1\01April_2010_Thursday_heute.avi_pid0_fn000006-0.png
C:\Users\LENO

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:11<00:03,  3.97s/it]

C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000000-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000001-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000002-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000003-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000004-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000005-0.png
C:\Users\LENOVO\Project Final Year\dataset/train/01April_2010_Thursday_heute_default-4/1\01April_2010_Thursday_heute.avi_pid0_fn000006-0.png
C:\Users\LENO

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.55s/it]
